In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
airbnb=pd.read_csv('Airbnb_cleaned.csv',index_col=False)

In [3]:
airbnb.drop(['Unnamed: 0'],axis=1,inplace=True)

In [4]:
airbnb.head(1)

,id,host_id,borough,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,description,accommodates,zipcode
0,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,10,48,0.39,1,1,"Beautiful, spacious skylit studio in the heart...",1,10018.0


In [5]:
airbnb.isnull().sum()

id                                0
host_id                           0
borough                           0
neighbourhood_group               0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
description                       0
accommodates                      0
zipcode                           1
dtype: int64

In [6]:
airbnb.dropna(inplace=True)

In [7]:
airbnb.dtypes

id                                  int64
host_id                             int64
borough                            object
neighbourhood_group                object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
description                        object
accommodates                        int64
zipcode                           float64
dtype: object

In [8]:
airbnb.zipcode = airbnb.zipcode.astype(np.int64)

In [9]:
yelp=pd.read_csv('yelp_cleaned.csv',index_col=False)
yelp.drop(['Unnamed: 0'],axis=1,inplace=True)

In [10]:
yelp.head(1)

,id,name,is_closed,review_count,category alias,category title,rating,latitude,longitude,distance,price,zipcode,borough,neighborhood
0,33tYNCEBSlDaQOgRq8-L9g,Muse Bar,False,6,cocktailbars,Cocktail Bars,5.0,40.75792,-73.98375,520.110129,2.0,10036,Manhattan,Chelsea and Clinton


In [11]:
yelp.dtypes

id                 object
name               object
is_closed            bool
review_count        int64
category alias     object
category title     object
rating            float64
latitude          float64
longitude         float64
distance          float64
price             float64
zipcode             int64
borough            object
neighborhood       object
dtype: object

In [12]:
yelp.isnull().sum()

id                0
name              0
is_closed         0
review_count      0
category alias    0
category title    0
rating            0
latitude          0
longitude         0
distance          0
price             0
zipcode           0
borough           0
neighborhood      0
dtype: int64

In [13]:
#yelp.zipcode.sort_values(ascending=False)

In [14]:
# let's keep New Jersey data apart

In [15]:
yelp=yelp[yelp['borough']!='New Jersey']

In [16]:
yelp.head(1)

,id,name,is_closed,review_count,category alias,category title,rating,latitude,longitude,distance,price,zipcode,borough,neighborhood
0,33tYNCEBSlDaQOgRq8-L9g,Muse Bar,False,6,cocktailbars,Cocktail Bars,5.0,40.75792,-73.98375,520.110129,2.0,10036,Manhattan,Chelsea and Clinton


In [17]:
# Merge Airbnb and Yelp

In [18]:
yelp['category title'][:5]

0    Cocktail Bars
1             Bars
2          Lounges
3    Cocktail Bars
4        Wine Bars
Name: category title, dtype: object

In [19]:
categories = []
for iter in yelp['category title']:
    categories.extend(str.split(str(iter), sep=','))

categories = pd.Series(categories)
categories = categories.str.lstrip()
categories.value_counts()

Coffee & Tea              1008
Bars                       535
Italian                    400
Cafes                      343
Cocktail Bars              294
American (New)             280
Mexican                    211
Lounges                    190
Pizza                      187
Bakeries                   163
Pubs                       161
American (Traditional)     146
Seafood                    145
Sports Bars                137
Japanese                   124
Korean                     118
Thai                       118
Chinese                    117
Wine Bars                  104
Dive Bars                  101
Caribbean                   95
Breakfast & Brunch          91
Latin American              88
French                      88
Mediterranean               87
Indian                      78
Burgers                     72
Desserts                    60
Sushi Bars                  58
Greek                       57
                          ... 
Mongolian                    1
Cheese S

In [20]:
def unpack_categories(df, column):
    for index, value in df[[column]].iterrows():
        df.loc[index, 'is_restaurant'] = ('Bars' not in str(value)) 
        df.loc[index, 'is_bar'] = ('Bars' in str(value)) | ('Nightlife' in str(value))
    return df

In [21]:
yelp_df_copy = yelp.copy()

In [22]:
yelp_df_copy = unpack_categories(yelp_df_copy, 'category title')

In [23]:
yelp_df_copy.is_restaurant.value_counts()

True     6166
False    1372
Name: is_restaurant, dtype: int64

In [31]:
yelp_df_copy.head()

,id,name,is_closed,review_count,category alias,category title,rating,latitude,longitude,distance,price,zipcode,borough,neighborhood,is_restaurant,is_bar
0,33tYNCEBSlDaQOgRq8-L9g,Muse Bar,False,6,cocktailbars,Cocktail Bars,5.0,40.757920,-73.983750,520.110129,2.0,10036,Manhattan,Chelsea and Clinton,False,True
1,rVwQcrogYw-O2SsAk51iYA,Bemelmans Bar,False,302,bars,Bars,4.0,40.774379,-73.963308,2029.144814,3.0,10021,Manhattan,Upper East Side,False,True
2,fAu6e8uCg1lfqIQ1PKNZyA,Tipsy NoMad,False,13,lounges,Lounges,4.5,40.743772,-73.984627,1891.642400,2.0,10016,Manhattan,Gramercy Park and Murray Hill,True,False
3,JZ06I46tmJlvYA_NZ7zyxw,The Skylark,False,559,cocktailbars,Cocktail Bars,4.0,40.754295,-73.989000,1102.456621,3.0,10018,Manhattan,Chelsea and Clinton,False,True
4,mvAjuaS1Jp6dsN4a-i_Fyw,Pocket Bar NYC,False,287,wine_bars,Wine Bars,4.5,40.763365,-73.992299,1213.736069,2.0,10019,Manhattan,Chelsea and Clinton,False,True


In [32]:
print(yelp_df_copy.shape)
print(airbnb.shape)

(7538, 16)
(49110, 16)


In [33]:
airbnb.columns

Index(['id', 'host_id', 'borough', 'neighbourhood_group', 'latitude',
       'longitude', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365', 'description',
       'accommodates', 'zipcode'],
      dtype='object')

In [34]:
zip_to_bor = pd.read_csv('neighborhood-NYC.csv')
borough_dict = dict(zip(zip_to_bor['zipcode'], zip_to_bor['neighborhood']))
airbnb['neighborhood'] = airbnb['zipcode'].map(borough_dict)

In [35]:
yelp_df_copy.columns

Index(['id', 'name', 'is_closed', 'review_count', 'category alias',
       'category title', 'rating', 'latitude', 'longitude', 'distance',
       'price', 'zipcode', 'borough', 'neighborhood', 'is_restaurant',
       'is_bar'],
      dtype='object')

In [30]:
# We needto merge data on neighborhood/zipcode/lat/long so let's start with neighborhood

In [36]:
airbnb_nhood=pd.DataFrame(airbnb['neighborhood'].value_counts().reset_index())

In [37]:
airbnb_nhood.head()

,index,neighborhood
0,Bushwick and Williamsburg,4483
1,Central Brooklyn,4158
2,Chelsea and Clinton,4079
3,Lower East Side,3467
4,Greenpoint,3289


In [38]:
airbnb_nhood.isnull().sum()

index           0
neighborhood    0
dtype: int64

In [39]:
yelp_nhood = pd.DataFrame(yelp_df_copy['neighborhood'].value_counts().reset_index())

In [40]:
yelp_nhood.head()

,index,neighborhood
0,Chelsea and Clinton,719
1,Northwest Brooklyn,615
2,Northwest Queens,473
3,Greenwich Village and Soho,457
4,Lower East Side,449


In [41]:
print(yelp_nhood.shape)
print(airbnb_nhood.shape)

(44, 2)
(42, 2)


In [42]:
match_hoods = pd.merge(airbnb_nhood, yelp_nhood,on=['index'], how='outer',suffixes=('_airbnb', '_yelp'))

In [43]:
match_hoods.head()

,index,neighborhood_airbnb,neighborhood_yelp
0,Bushwick and Williamsburg,4483.0,254
1,Central Brooklyn,4158.0,324
2,Chelsea and Clinton,4079.0,719
3,Lower East Side,3467.0,449
4,Greenpoint,3289.0,317


In [44]:
#Count records where there is no match on each side.
airbnb_nomatch = match_hoods.neighborhood_airbnb[match_hoods.neighborhood_yelp.isnull()]
print('There are ' + str(airbnb_nomatch.sum()) + ' AirBnB listings with no match in the Yelp data.')

yelp_nomatch = match_hoods.neighborhood_yelp[match_hoods.neighborhood_airbnb.isnull()]
print('There are ' + str(yelp_nomatch.sum()) + ' Yelp listings with no match in the AirBnB data.')

There are 0.0 AirBnB listings with no match in the Yelp data.
There are 106 Yelp listings with no match in the AirBnB data.


In [68]:
print(match_hoods.sort_values('index'))

                            index  neighborhood_airbnb  neighborhood_yelp
19                   Borough Park                589.0                 83
33         Bronx Park and Fordham                181.0                118
0       Bushwick and Williamsburg               4483.0                254
27         Canarsie and Flatlands                254.0                 29
35                  Central Bronx                120.0                 34
1                Central Brooklyn               4158.0                324
7                  Central Harlem               2066.0                104
36                 Central Queens                105.0                 24
2             Chelsea and Clinton               4079.0                719
15                    East Harlem               1056.0                 51
18     East New York and New Lots                755.0                 22
13                       Flatbush               1689.0                129
6   Gramercy Park and Murray Hill     

In [45]:
# We are going to recommend listings based of rest,safety.. but if there are no listings in that area we cannot reommend it..
# so here I am just going to drop Yonkers and New Rochelle from Yelp ..

In [46]:
#to_drop=['Yonkers','New Rochelle']
#match_hoods[~match_hoods['neighborhood_yelp'].isin(to_drop)]

In [47]:
match_hoods=match_hoods.dropna()

In [48]:
airbnb_nomatch = match_hoods.neighborhood_airbnb[match_hoods.neighborhood_yelp.isnull()]
print('There are ' + str(airbnb_nomatch.sum()) + ' AirBnB listings with no match in the Yelp data.')

yelp_nomatch = match_hoods.neighborhood_yelp[match_hoods.neighborhood_airbnb.isnull()]
print('There are ' + str(yelp_nomatch.sum()) + ' Yelp listings with no match in the AirBnB data.')

There are 0.0 AirBnB listings with no match in the Yelp data.
There are 0 Yelp listings with no match in the AirBnB data.


In [49]:
yelp_df_copy.head()

,id,name,is_closed,review_count,category alias,category title,rating,latitude,longitude,distance,price,zipcode,borough,neighborhood,is_restaurant,is_bar
0,33tYNCEBSlDaQOgRq8-L9g,Muse Bar,False,6,cocktailbars,Cocktail Bars,5.0,40.757920,-73.983750,520.110129,2.0,10036,Manhattan,Chelsea and Clinton,False,True
1,rVwQcrogYw-O2SsAk51iYA,Bemelmans Bar,False,302,bars,Bars,4.0,40.774379,-73.963308,2029.144814,3.0,10021,Manhattan,Upper East Side,False,True
2,fAu6e8uCg1lfqIQ1PKNZyA,Tipsy NoMad,False,13,lounges,Lounges,4.5,40.743772,-73.984627,1891.642400,2.0,10016,Manhattan,Gramercy Park and Murray Hill,True,False
3,JZ06I46tmJlvYA_NZ7zyxw,The Skylark,False,559,cocktailbars,Cocktail Bars,4.0,40.754295,-73.989000,1102.456621,3.0,10018,Manhattan,Chelsea and Clinton,False,True
4,mvAjuaS1Jp6dsN4a-i_Fyw,Pocket Bar NYC,False,287,wine_bars,Wine Bars,4.5,40.763365,-73.992299,1213.736069,2.0,10019,Manhattan,Chelsea and Clinton,False,True


In [50]:
yelp_df_copy.neighborhood.value_counts()

Chelsea and Clinton              719
Northwest Brooklyn               615
Northwest Queens                 473
Greenwich Village and Soho       457
Lower East Side                  449
Gramercy Park and Murray Hill    372
Central Brooklyn                 324
Greenpoint                       317
West Queens                      305
Bushwick and Williamsburg        254
North Queens                     233
West Central Queens              233
Southeast Bronx                  225
South Shore                      175
Upper East Side                  168
Upper West Side                  164
Lower Manhattan                  156
Stapleton and St. George         153
Flatbush                         129
Inwood and Washington Heights    127
Bronx Park and Fordham           118
Central Harlem                   104
Mid-Island                        98
Southern Brooklyn                 88
Southwest Brooklyn                87
High Bridge and Morrisania        84
Borough Park                      83
P

In [51]:
yelp_df_copy=yelp_df_copy[~(yelp_df_copy.neighborhood.isin(["Yonkers", "New Rochelle"]))]

In [52]:
yelp_df_copy.neighborhood.value_counts()

Chelsea and Clinton              719
Northwest Brooklyn               615
Northwest Queens                 473
Greenwich Village and Soho       457
Lower East Side                  449
Gramercy Park and Murray Hill    372
Central Brooklyn                 324
Greenpoint                       317
West Queens                      305
Bushwick and Williamsburg        254
West Central Queens              233
North Queens                     233
Southeast Bronx                  225
South Shore                      175
Upper East Side                  168
Upper West Side                  164
Lower Manhattan                  156
Stapleton and St. George         153
Flatbush                         129
Inwood and Washington Heights    127
Bronx Park and Fordham           118
Central Harlem                   104
Mid-Island                        98
Southern Brooklyn                 88
Southwest Brooklyn                87
High Bridge and Morrisania        84
Borough Park                      83
P

In [53]:
airbnb.columns

Index(['id', 'host_id', 'borough', 'neighbourhood_group', 'latitude',
       'longitude', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365', 'description',
       'accommodates', 'zipcode', 'neighborhood'],
      dtype='object')

In [55]:
yelp_df_copy.columns

Index(['id', 'name', 'is_closed', 'review_count', 'category alias',
       'category title', 'rating', 'latitude', 'longitude', 'distance',
       'price', 'zipcode', 'borough', 'neighborhood', 'is_restaurant',
       'is_bar'],
      dtype='object')

In [96]:
#airbnb= airbnb.rename(columns={'neighbourhood':'neighborhood', 'latitude':'latitude', 
 #                         'longitude_x':'longitude'})

In [56]:
#hAVERSINE: https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
from math import sin, cos, sqrt, atan2, radians

def compute_distance(lat1, lon1, lat2, lon2):

    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

In [57]:
yelp_restaurants=yelp_df_copy.loc[yelp_df_copy['is_restaurant'] == True]

In [58]:
yelp_restaurants.drop(['is_bar'],axis=1,inplace=True)

C:\Apps\HarshD\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [60]:
yelp_bar=yelp_df_copy.loc[yelp_df_copy['is_bar'] == True]
yelp_bar.drop(['is_restaurant'],axis=1,inplace=True)

C:\Apps\HarshD\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [59]:
a_hoods = airbnb[['id', 'neighborhood', 'latitude', 'longitude']]
a_hoods.head()

,id,neighborhood,latitude,longitude
0,2595,Chelsea and Clinton,40.75362,-73.98377
1,3831,Central Brooklyn,40.68514,-73.95976
2,5099,Gramercy Park and Murray Hill,40.74767,-73.97500
3,5121,Central Brooklyn,40.68688,-73.95596
4,5178,Chelsea and Clinton,40.76489,-73.98493


In [63]:
y_hoods_r = yelp_restaurants[['id', 'neighborhood', 'latitude', 'longitude','review_count','rating']]
y_hoods_r.head()

,id,neighborhood,latitude,longitude,review_count,rating
2,fAu6e8uCg1lfqIQ1PKNZyA,Gramercy Park and Murray Hill,40.743772,-73.984627,13,4.5
7,1hScmfYV42XshcprL79MAQ,Gramercy Park and Murray Hill,40.749948,-73.976572,206,4.0
9,mn6JM5lHuSSsG9KSXck3ww,Lower Manhattan,40.704280,-74.009919,754,3.0
11,tiwrAjwddw-3TSrnWSYD7w,Gramercy Park and Murray Hill,40.750580,-73.980180,346,4.0
18,Ea2kgAIGHoVQA8t9wL-RKw,Greenwich Village and Soho,40.719240,-74.000160,73,3.0


In [65]:
y_hoods_b = yelp_bar[['id', 'neighborhood', 'latitude', 'longitude','review_count','rating']]
y_hoods_b.head()

,id,neighborhood,latitude,longitude,review_count,rating
0,33tYNCEBSlDaQOgRq8-L9g,Chelsea and Clinton,40.757920,-73.983750,6,5.0
1,rVwQcrogYw-O2SsAk51iYA,Upper East Side,40.774379,-73.963308,302,4.0
3,JZ06I46tmJlvYA_NZ7zyxw,Chelsea and Clinton,40.754295,-73.989000,559,4.0
4,mvAjuaS1Jp6dsN4a-i_Fyw,Chelsea and Clinton,40.763365,-73.992299,287,4.5
5,GRhA1hH1edG48tj2T5I6CA,Gramercy Park and Murray Hill,40.758074,-73.968724,149,4.0


In [66]:
restaurants = pd.merge(a_hoods, y_hoods_r, how='inner', on='neighborhood', suffixes = ('_airbnb', '_yelp'))
restaurants.shape

(12002757, 9)

In [67]:
restaurants['distance'] = list(map(compute_distance, restaurants['latitude_airbnb'], 
                            restaurants['longitude_airbnb'], restaurants['latitude_yelp'],
                            restaurants['longitude_yelp']))

In [68]:
restaurants.head()

,id_airbnb,neighborhood,latitude_airbnb,longitude_airbnb,id_yelp,latitude_yelp,longitude_yelp,review_count,rating,distance
0,2595,Chelsea and Clinton,40.75362,-73.98377,gnp2CQMuLlz8eXXbKaOY5g,40.767600,-73.983130,7,4.0,1.555928
1,2595,Chelsea and Clinton,40.75362,-73.98377,HZoA3-oZiS3DbMonlgclAw,40.760980,-73.977313,38,4.0,0.982932
2,2595,Chelsea and Clinton,40.75362,-73.98377,vZE7WQQZaxQRJ0eOaAAItQ,40.758715,-73.988284,65,4.0,0.682505
3,2595,Chelsea and Clinton,40.75362,-73.98377,4WDIwsupzB5XNdjFO_plQg,40.763595,-73.992218,91,4.0,1.318197
4,2595,Chelsea and Clinton,40.75362,-73.98377,ahMxvJI5SnqHUccFSpYWwg,40.750540,-73.990550,116,4.0,0.666138


In [82]:
#filter for distance < .1
restaurants_1 = restaurants[restaurants['distance']<=.1]
#filter for distance < .5
restaurants_5 =restaurants[restaurants['distance']<=.5]
print(restaurants_1.shape)
print(restaurants_5.shape)

(74256, 10)
(1389958, 10)


In [83]:
#For everything within .1 mile:
restaurants_1mile = restaurants_1.groupby(['id_airbnb']).agg({'id_yelp':'count',
                                                        'review_count':'sum',
                                                        'rating':'mean'})

In [84]:
restaurants_1mile.head()

,id_yelp,review_count,rating
id_airbnb,,,
2595,4,964,3.875
5099,1,759,4.500
5121,2,163,3.750
5178,1,88,3.500
5203,1,2502,4.000


In [85]:
restaurants_1mile  = restaurants_1mile.rename(columns={'id_yelp':'total_restaurants_1', 
                                          'review_count':'total_restaurants_reviews_1', 
                                          'rating':'restaurants_avg_rating_1'})
restaurants_1mile.head()

,total_restaurants_1,total_restaurants_reviews_1,restaurants_avg_rating_1
id_airbnb,,,
2595,4,964,3.875
5099,1,759,4.500
5121,2,163,3.750
5178,1,88,3.500
5203,1,2502,4.000


In [80]:
#For everything within .1 mile:
restaurants_5mile = restaurants_5.groupby(['id_airbnb']).agg({'id_yelp':'count',
                                                        'review_count':'sum',
                                                        'rating':'mean'})
restaurants_5mile = restaurants_5mile.rename(columns={'id_yelp':'restaurants_yelp_5', 
                                          'review_count':'total_restaurants_reviews_5', 
                                          'rating':'restaurants_avg_rating_5'})

In [81]:
restaurants_5mile.head()

,restaurants_yelp_5,total_restaurants_reviews_5,restaurants_avg_rating_5
id_airbnb,,,
2595,78,40022,4.057692
3831,27,4086,4.240741
5099,42,15610,4.166667
5121,17,2519,3.970588
5178,94,61348,4.037234


In [86]:
all_restaurants = pd.merge(restaurants_1mile , restaurants_5mile , how='right', on='id_airbnb')
all_restaurants.head()

,total_restaurants_1,total_restaurants_reviews_1,restaurants_avg_rating_1,restaurants_yelp_5,total_restaurants_reviews_5,restaurants_avg_rating_5
id_airbnb,,,,,,
2595,4.0,964.0,3.875,78,40022,4.057692
5099,1.0,759.0,4.500,42,15610,4.166667
5121,2.0,163.0,3.750,17,2519,3.970588
5178,1.0,88.0,3.500,94,61348,4.037234
5203,1.0,2502.0,4.000,30,10201,4.050000


In [88]:
bars = pd.merge(a_hoods, y_hoods_b, how='inner', on='neighborhood', suffixes = ('_airbnb', '_yelp'))
#print(bars.shape)
bars['distance'] = list(map(compute_distance, bars['latitude_airbnb'], 
                            bars['longitude_airbnb'], bars['latitude_yelp'],
                            bars['longitude_yelp']))
#print(bars.head())
#filter for distance < .1
bars_1 = bars[bars['distance']<=.1]
#filter for distance < .5
bars_5 =bars[bars['distance']<=.5]
#print(bars_1.shape)
#print(bars_5.shape)
#For everything within .1 mile:
bars_1mile = bars_1.groupby(['id_airbnb']).agg({'id_yelp':'count',
                                                        'review_count':'sum',
                                                        'rating':'mean'})
bars_1mile.head()
bars_1mile  = bars_1mile.rename(columns={'id_yelp':'total_bars_1', 
                                          'review_count':'total_bars_reviews_1', 
                                          'rating':'bars_avg_rating_1'})
#print(bars_1mile.head())
#For everything within .1 mile:
bars_5mile = bars_5.groupby(['id_airbnb']).agg({'id_yelp':'count',
                                                        'review_count':'sum',
                                                        'rating':'mean'})
bars_5mile = bars_5mile.rename(columns={'id_yelp':'bars_yelp_5', 
                                          'review_count':'total_bars_reviews_5', 
                                          'rating':'bars_avg_rating_5'})
all_bars = pd.merge(bars_1mile , bars_5mile , how='right', on='id_airbnb')
print(all_bars.head())

           total_bars_1  total_bars_reviews_1  bars_avg_rating_1  bars_yelp_5  \
id_airbnb                                                                       
2595                1.0                  10.0                4.5           21   
5121                1.0                  61.0                4.5            7   
5178                1.0                 428.0                4.0           24   
5441                1.0                1565.0                4.5           30   
5803                1.0                 101.0                4.0           11   

           total_bars_reviews_5  bars_avg_rating_5  
id_airbnb                                           
2595                       8701           4.000000  
5121                        599           4.142857  
5178                      10378           4.083333  
5441                      10518           4.200000  
5803                       1620           4.090909  


In [89]:
print(all_restaurants.shape)
print(all_bars.shape)

(46275, 6)
(41528, 6)


In [91]:
rest_df=pd.merge(all_restaurants, all_bars, on='id_airbnb', how='left')

In [92]:
rest_df.head()

,total_restaurants_1,total_restaurants_reviews_1,restaurants_avg_rating_1,restaurants_yelp_5,total_restaurants_reviews_5,restaurants_avg_rating_5,total_bars_1,total_bars_reviews_1,bars_avg_rating_1,bars_yelp_5,total_bars_reviews_5,bars_avg_rating_5
id_airbnb,,,,,,,,,,,,
2595,4.0,964.0,3.875,78,40022,4.057692,1.0,10.0,4.5,21.0,8701.0,4.000000
5099,1.0,759.0,4.500,42,15610,4.166667,NaN,NaN,NaN,6.0,2406.0,4.083333
5121,2.0,163.0,3.750,17,2519,3.970588,1.0,61.0,4.5,7.0,599.0,4.142857
5178,1.0,88.0,3.500,94,61348,4.037234,1.0,428.0,4.0,24.0,10378.0,4.083333
5203,1.0,2502.0,4.000,30,10201,4.050000,NaN,NaN,NaN,7.0,1027.0,4.071429


In [95]:
airbnb=airbnb.rename(columns={'id':'id_airbnb'})

In [94]:
#airbnb.head()

In [96]:
airbnb_df=pd.merge(airbnb, rest_df, on='id_airbnb', how='left')
airbnb_df.head(2)

,id_airbnb,host_id,borough,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,description,accommodates,zipcode,neighborhood,total_restaurants_1,total_restaurants_reviews_1,restaurants_avg_rating_1,restaurants_yelp_5,total_restaurants_reviews_5,restaurants_avg_rating_5,total_bars_1,total_bars_reviews_1,bars_avg_rating_1,bars_yelp_5,total_bars_reviews_5,bars_avg_rating_5
0,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,10,48,0.39,1,1,"Beautiful, spacious skylit studio in the heart...",1,10018,Chelsea and Clinton,4.0,964.0,3.875,78.0,40022.0,4.057692,1.0,10.0,4.5,21.0,8701.0,4.000000
1,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,295,4.67,1,1,Urban retreat: enjoy 500 s.f. floor in 1899 br...,3,11238,Central Brooklyn,NaN,NaN,NaN,27.0,4086.0,4.240741,NaN,NaN,NaN,12.0,790.0,4.208333


In [97]:
airbnb_df.describe()

,id_airbnb,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,accommodates,zipcode,total_restaurants_1,total_restaurants_reviews_1,restaurants_avg_rating_1,restaurants_yelp_5,total_restaurants_reviews_5,restaurants_avg_rating_5,total_bars_1,total_bars_reviews_1,bars_avg_rating_1,bars_yelp_5,total_bars_reviews_5,bars_avg_rating_5
count,4.911000e+04,4.911000e+04,49110.000000,49110.000000,49110.000000,49110.000000,49110.000000,49110.000000,49110.000000,49110.000000,49110.000000,4.911000e+04,24303.000000,24303.000000,24303.000000,46275.000000,46275.000000,46275.000000,12567.000000,12567.000000,12567.000000,41323.000000,41323.000000,41323.000000
mean,2.112829e+07,7.784788e+07,40.728802,-73.951039,158.773346,7.578782,25.262492,1.130640,7.481450,112.677092,2.871045,4.252803e+04,3.055425,1312.247583,4.119992,30.036910,12651.782366,4.111260,1.625527,345.529641,4.058002,9.429809,1987.279409,4.082483
std,1.238693e+07,9.043076e+07,0.054846,0.047280,351.859956,22.545924,47.766811,1.622962,33.229342,135.876147,1.905804,5.172036e+06,2.608539,2140.830303,0.365266,27.828889,18451.861337,0.213732,0.976720,471.700146,0.450313,9.089482,2810.609069,0.313111
min,2.595000e+03,2.438000e+03,40.499790,-74.247870,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,7.093000e+03,1.000000,1.000000,1.000000,1.000000,1.000000,1.500000,1.000000,1.000000,2.000000,1.000000,1.000000,2.000000
25%,9.977018e+06,8.700077e+06,40.689650,-73.982908,69.000000,1.000000,1.000000,0.040000,1.000000,0.000000,2.000000,1.002400e+04,1.000000,136.000000,4.000000,9.000000,1062.000000,4.005263,1.000000,65.000000,3.875000,3.000000,275.000000,3.937500
50%,2.126653e+07,3.485137e+07,40.722930,-73.955170,106.000000,3.000000,6.000000,0.380000,1.000000,41.000000,2.000000,1.110500e+04,2.000000,495.000000,4.062500,21.000000,4533.000000,4.102273,1.000000,172.000000,4.000000,7.000000,858.000000,4.071429
75%,3.239957e+07,1.268974e+08,40.763000,-73.934670,175.000000,5.000000,26.000000,1.670000,2.000000,226.000000,4.000000,1.122100e+04,4.000000,1507.000000,4.333333,43.000000,15609.000000,4.212121,2.000000,433.000000,4.500000,13.000000,2382.000000,4.250000
max,4.058402e+07,3.143682e+08,40.916860,-73.712990,10000.000000,1250.000000,675.000000,52.630000,310.000000,365.000000,25.000000,1.124911e+09,26.000000,20440.000000,5.000000,135.000000,90850.000000,5.000000,8.000000,4845.000000,5.000000,54.000000,15550.000000,5.000000


In [98]:
airbnb_df.to_csv('airbnb_merge.csv')

In [99]:
airbnb_df.head()

,id_airbnb,host_id,borough,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,description,accommodates,zipcode,neighborhood,total_restaurants_1,total_restaurants_reviews_1,restaurants_avg_rating_1,restaurants_yelp_5,total_restaurants_reviews_5,restaurants_avg_rating_5,total_bars_1,total_bars_reviews_1,bars_avg_rating_1,bars_yelp_5,total_bars_reviews_5,bars_avg_rating_5
0,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,10,48,0.39,1,1,"Beautiful, spacious skylit studio in the heart...",1,10018,Chelsea and Clinton,4.0,964.0,3.875,78.0,40022.0,4.057692,1.0,10.0,4.5,21.0,8701.0,4.000000
1,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,295,4.67,1,1,Urban retreat: enjoy 500 s.f. floor in 1899 br...,3,11238,Central Brooklyn,NaN,NaN,NaN,27.0,4086.0,4.240741,NaN,NaN,NaN,12.0,790.0,4.208333
2,5099,7322,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,78,0.60,1,19,My large 1 bedroom apartment has a true New Yo...,2,10016,Gramercy Park and Murray Hill,1.0,759.0,4.500,42.0,15610.0,4.166667,NaN,NaN,NaN,6.0,2406.0,4.083333
3,5121,7356,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,60,29,49,0.38,1,365,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...,2,11216,Central Brooklyn,2.0,163.0,3.750,17.0,2519.0,3.970588,1.0,61.0,4.5,7.0,599.0,4.142857
4,5178,8967,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,2,454,3.52,1,242,Please don’t expect the luxury here just a bas...,2,10019,Chelsea and Clinton,1.0,88.0,3.500,94.0,61348.0,4.037234,1.0,428.0,4.0,24.0,10378.0,4.083333


In [104]:
subway=pd.read_csv('airbnb_w_station.csv')

In [106]:
subway.head(1)

,Unnamed: 0,Unnamed: 0.1,id_airbnb,host_id,borough,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,description,accommodates,zipcode,neighborhood,total_yelp_1,yelp_total_reviews_1,yelp_avg_rating_1,total_yelp_5,yelp_total_reviews_5,yelp_avg_rating_5,nearest_station,lat,lon,distance_to_station
0,0,0,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,10,48,0.39,1,1,"Beautiful, spacious skylit studio in the heart...",1,10018,Chelsea and Clinton,5.0,974.0,4.0,99.0,48723.0,4.045455,42nd St - Bryant Pk,40.754184,-73.984591,0.093386


In [107]:
subway=subway[['id_airbnb','nearest_station','distance_to_station']]

In [108]:
final=pd.merge(airbnb_df,subway,on='id_airbnb')

In [110]:
final.head(2)

,id_airbnb,host_id,borough,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,description,accommodates,zipcode,neighborhood,total_restaurants_1,total_restaurants_reviews_1,restaurants_avg_rating_1,restaurants_yelp_5,total_restaurants_reviews_5,restaurants_avg_rating_5,total_bars_1,total_bars_reviews_1,bars_avg_rating_1,bars_yelp_5,total_bars_reviews_5,bars_avg_rating_5,nearest_station,distance_to_station
0,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,10,48,0.39,1,1,"Beautiful, spacious skylit studio in the heart...",1,10018,Chelsea and Clinton,4.0,964.0,3.875,78.0,40022.0,4.057692,1.0,10.0,4.5,21.0,8701.0,4.000000,42nd St - Bryant Pk,0.093386
1,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,295,4.67,1,1,Urban retreat: enjoy 500 s.f. floor in 1899 br...,3,11238,Central Brooklyn,NaN,NaN,NaN,27.0,4086.0,4.240741,NaN,NaN,NaN,12.0,790.0,4.208333,Classon Ave,0.417452


In [133]:
final.isnull().sum()

id_airbnb                             0
host_id                               0
borough                               0
neighbourhood_group                   0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                     0
calculated_host_listings_count        0
availability_365                      0
description                           0
accommodates                          0
zipcode                               0
neighborhood                          0
total_restaurants_1               24807
total_restaurants_reviews_1       24807
restaurants_avg_rating_1          24807
restaurants_yelp_5                 2835
total_restaurants_reviews_5        2835
restaurants_avg_rating_5           2835
total_bars_1                      36543
total_bars_reviews_1              36543


In [126]:
zip_to_bor = pd.read_csv('neighborhood-NYC.csv')
borough_dict_a = dict(zip(zip_to_bor['zipcode'], zip_to_bor['neighborhood']))
final['neighborhood'] = final['zipcode'].map(borough_dict_a)

In [128]:
#final.neighborhood.value_counts()

In [124]:
print(final[final["neighborhood"].isnull()][['borough','zipcode']])

         borough     zipcode
3745    Brooklyn  1124911249
8439      Queens   111033233
9330   Manhattan   100022289
10143     Queens       11001
17062     Queens   113852308
21676  Manhattan       10281
21753     Queens       91766
24098     Queens   114133220
24806  Manhattan        7093
29049     Queens       11559
31496  Manhattan       10281
38916  Manhattan       10270
39300  Manhattan       10270
39764     Queens       11001
43040     Queens       11559
46471      Bronx       10705
47292      Bronx       10705


In [125]:
print(final[final["neighborhood"].isnull()]['zipcode'].value_counts())

10270         2
11001         2
10705         2
10281         2
11559         2
100022289     1
91766         1
7093          1
113852308     1
1124911249    1
111033233     1
114133220     1
Name: zipcode, dtype: int64


In [129]:
# let's fill the na in neighborhood with borough values

In [132]:
final['neighborhood'].fillna(final['borough'],inplace=True)

In [135]:
final.to_csv('Airbnb_yelp_subway.csv')

In [112]:
#merge.to_pickle('full_merge.pickle')
#merge.loc[merge['distance']>0].head()